In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_4D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2), x[1])
    theta3 = np.arctan2(x[3], x[2])
    
    
    return rt+ndt, (theta1, theta2, theta3)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            if a(j*dt) == 0:
                continue
            
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_4D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0] - prms[1]*t
    a2 = lambda t: (a(t))**2
    da2 = lambda t: -2*prms[1] * a(t)
    
    ndt = prms[2]
    mu = np.array([prms[3], prms[4], prms[5], prms[6]])
    
    T_max = min(max(RT), prms[0]/prms[1])
    fpt = ie_bessel_fpt(a2, da2, mu.shape[0], 0.000001, 
                        dt=0.02, T_max=T_max)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.0001 and rt - ndt < T_max:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])*np.cos(theta[2])
            mu_dot_x3 = mu[3]*np.sin(theta[0])*np.sin(theta[1])*np.sin(theta[2]) 
            term1 = a(rt - ndt) * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2 + mu_dot_x3)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'b0_true': [],
               'b0_estimate': [],
               'lambda_true': [],
               'lambda_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': [],
               'mu4_true': [],
               'mu4_estimate': []}

min_b0 = 2
max_b0 = 5

min_lambda = .1
max_lambda = 2

min_ndt = 0.1
max_ndt = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(50)):
    b0 = np.random.uniform(min_b0, max_b0)
    lamb = np.random.uniform(min_lambda, max_lambda)
    a = lambda t: b0 - lamb*t
    
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(500):
        rt, theta = simulate_HSDM_4D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_4D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                                             (min_ndt, max_ndt), (min_mu, max_mu), 
                                             (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])
    
    min_ans = minimize(HSDM_4D_likelihood,
                       args=(RT, Theta),
                       method='nelder-mead',
                       x0=min_ans.x,
                       bounds=[(min_b0, max_b0), (min_lambda, max_lambda), (min_ndt, max_ndt),
                                (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])    
    
    if min_ans.success:
        recovery_df['b0_true'].append(b0)
        recovery_df['lambda_true'].append(lamb)
        recovery_df['ndt_true'].append(ndt)
        recovery_df['mu1_true'].append(mu[0])
        recovery_df['mu2_true'].append(mu[1])
        recovery_df['mu3_true'].append(mu[2])
        recovery_df['mu4_true'].append(mu[3])

        recovery_df['b0_estimate'].append(min_ans.x[0])
        recovery_df['lambda_estimate'].append(min_ans.x[1])
        recovery_df['ndt_estimate'].append(min_ans.x[2])
        recovery_df['mu1_estimate'].append(min_ans.x[3])
        recovery_df['mu2_estimate'].append(min_ans.x[4])
        recovery_df['mu3_estimate'].append(min_ans.x[5])
        recovery_df['mu4_estimate'].append(min_ans.x[6])

100%|██████████████████████████████| 50/50 [3:21:43<00:00, 242.07s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_df)
recovery_df.corr()

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
b0_true,1.000000,0.903393,0.085897,-0.084810,-0.051752,0.001882,0.129307,0.129408,-0.041284,-0.053967,0.062628,0.063456,0.199804,0.206731
b0_estimate,0.903393,1.000000,-0.054619,0.038517,0.003899,-0.027033,0.087863,0.090855,0.021510,0.011060,0.015125,0.016935,0.175949,0.178136
lambda_true,0.085897,-0.054619,1.000000,0.787700,0.130294,0.197580,0.130049,0.132798,-0.071739,-0.073723,0.170368,0.169603,-0.084228,-0.075472
lambda_estimate,-0.084810,0.038517,0.787700,1.000000,0.194842,0.157088,0.041166,0.048769,0.003482,0.005265,0.103549,0.105743,-0.161680,-0.156698
ndt_true,-0.051752,0.003899,0.130294,0.194842,1.000000,0.979010,-0.330836,-0.331123,-0.137766,-0.145879,-0.061535,-0.068535,0.074560,0.072572
ndt_estimate,0.001882,-0.027033,0.197580,0.157088,0.979010,1.000000,-0.310731,-0.311920,-0.165259,-0.174702,-0.039825,-0.047119,0.073083,0.073833
mu1_true,0.129307,0.087863,0.130049,0.041166,-0.330836,-0.310731,1.000000,0.998886,0.005584,0.008436,-0.025102,-0.023014,-0.040388,-0.049691
mu1_estimate,0.129408,0.090855,0.132798,0.048769,-0.331123,-0.311920,0.998886,1.000000,0.008506,0.011936,-0.024361,-0.021919,-0.044985,-0.054869
mu2_true,-0.041284,0.021510,-0.071739,0.003482,-0.137766,-0.165259,0.005584,0.008506,1.000000,0.998788,0.136658,0.132599,-0.175612,-0.179004
mu2_estimate,-0.053967,0.011060,-0.073723,0.005265,-0.145879,-0.174702,0.008436,0.011936,0.998788,1.000000,0.136220,0.132094,-0.172954,-0.176528


In [8]:
file_name = '_Recovery_data/IE_4d_recovery_linear_500_02.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
0,3.422841,3.769044,0.815469,1.251666,0.287800,0.256546,-0.585328,-0.676373,-2.832156,-2.761936,-0.131006,-0.070862,2.997387,2.882531
1,2.814087,2.486088,1.685831,1.367489,0.313983,0.359983,1.244327,1.212591,-1.198178,-1.385737,-1.514009,-1.588650,-1.580492,-1.602869
2,4.570897,4.355990,1.367959,1.152510,0.156177,0.201790,2.754176,2.824698,1.789119,1.780158,1.285610,1.473571,-1.221456,-1.274268
3,2.280701,2.354089,1.906957,2.000000,0.113419,0.104908,2.910419,2.823183,-1.507368,-1.404064,2.704223,2.705331,0.775741,0.820171
4,2.192476,2.128044,1.772249,1.908600,0.973431,0.991255,0.609228,0.651968,-0.743107,-0.678064,-2.327720,-2.287195,-2.159586,-2.232033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,4.571948,5.000000,1.588504,1.798077,0.992445,0.937022,0.348386,0.298274,-2.131433,-2.115599,0.289046,0.334975,2.200765,2.203307
146,4.025757,3.813476,1.204879,1.080024,0.467801,0.500253,0.621412,0.600626,-0.506969,-0.459345,-2.666701,-2.684406,-0.408187,-0.452261
147,3.777796,3.440921,0.531999,0.212660,0.588675,0.636113,-1.067356,-1.060114,2.867075,2.933261,1.235490,1.217160,-1.752635,-1.776081
148,4.185538,4.796963,0.532885,1.083574,0.368838,0.309568,0.812473,0.739503,1.758188,1.665501,-1.834398,-1.728501,2.985658,2.814459
